In [1]:
simbolo_test = "BTG"
simbolo_validazione = "DHT"
n_simboli_addestramento = 2000
epochs = 100
batch_size = 32
optimizer = "adam"
#learning_rate = 0.001
n_timesteps = 60 # n. barre del periodo passato per la ricerca di pattern, inclusa ultima data disponibile
giorni_previsione = 10 # n. barre nel futuro di cui si desidera prevedere il prezzo

elenco_features = [
    "Close",
    "EMA_5", 
    "EMA_20", 
    "EMA_50",
    "Open",  
    "High",
    "Low",
    "Volume",
    "MACDh",
    "PSAR",
    "PSARaf",
    "SUPERT", 
    "TRIX",
    "ATR",
    "DM_OSC",
    "ADX"
]
elenco_targets = [
    "EMA_5",
    "EMA_20", 
    "EMA_50"    
]

col_features = {col: idx for idx, col in enumerate(elenco_features)}
col_targets = {col: idx for idx, col in enumerate(elenco_targets)}
n_features = len(col_features)
n_targets = len(col_targets)

def crea_modello(n_timesteps, giorni_previsione, n_targets, n_features):
        model = Sequential()

        # Layer LSTM iniziale
        model.add(LSTM(70, input_shape=(n_timesteps, n_features)))
        model.add(RepeatVector(giorni_previsione))

        # lstm-2
        model.add(LSTM(90, return_sequences=True,
                       kernel_regularizer=l2(4.153520011432108e-05)))
        model.add(Dropout(0.3))
        # lstm-3
        model.add(LSTM(130, return_sequences=True,
                       kernel_regularizer=l2(4.153520011432108e-05)))
        model.add(Dropout(0.3))
        # lstm-4
        model.add(LSTM(70, return_sequences=True,
                       kernel_regularizer=l2(4.153520011432108e-05)))
        model.add(Dropout(0.4))

        # Aggiunta di layer Dense
        model.add(TimeDistributed(Dense(70, activation='relu')))
        model.add(Dropout(0))

        model.add(TimeDistributed(Dense(n_targets, activation='linear')))

        model.compile(optimizer="adam",
                      loss='mean_absolute_percentage_error')

        return model

from sklearn.preprocessing import PowerTransformer
#from sklearn.preprocessing import RobustScaler
from sklearn.base import clone
scaler = PowerTransformer()
#scaler = RobustScaler()



In [2]:
print("Importa librerie", end="", flush=True)

#from keras.losses import Huber
#print(".", end="", flush=True)
import pandas as pd
print(".", end="", flush=True)
import yfinance as yf
print(".", end="", flush=True)
import numpy as np
print(".", end="", flush=True)
import tensorflow as tf
print(".", end="", flush=True)
import funzioni as fx
print(".", end="", flush=True)
from tensorflow.keras.layers import Dropout
print(".", end="", flush=True)
from tensorflow.keras.layers import RepeatVector
print(".", end="", flush=True)
from tensorflow.keras.layers import TimeDistributed
print(".", end="", flush=True)
from keras.regularizers import l2
print(".", end="", flush=True)
from tensorflow.keras import Sequential
print(".", end="", flush=True)
from tensorflow.keras.layers import LSTM, Dense
print(".", end="", flush=True)
#from tensorflow.keras.optimizers import Adam
#print(".", end="", flush=True)
#from sklearn.model_selection import train_test_split
#print(".", end="", flush=True)
from tensorflow.keras.models import load_model
print(".", end="", flush=True)
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
print(".", end="", flush=True)
import os
print("=", end="\n", flush=True)

print("Download lista ticker")
lista_ticker = pd.read_parquet("Tickers_De_Giro.parquet")
lista_ticker = lista_ticker.sample(frac=1).reset_index(drop=True)
lista_ticker = lista_ticker.loc[(lista_ticker["Ticker"] != simbolo_test) & (lista_ticker["Ticker"] != simbolo_validazione) & (lista_ticker["Categoria"] != "D"), :]

if os.path.exists('LSTM.keras'):
    print("Caricamento modello esistente")
    model = load_model("LSTM.keras")
else:
    print("Creazione modello")
    model = crea_modello(n_timesteps, giorni_previsione, n_targets, n_features)


Importa librerie.............=
Download lista ticker
Caricamento modello esistente


In [ ]:
print("Download dati ticker validazione")
ticker_val = yf.download(simbolo_validazione, start='2010-01-01', end='2023-12-31', progress=False)
ticker_val.index = ticker_val.index.date
print("Calcolo indicatori ticker validazione")
ticker_val = fx.crea_indicatori(ticker_val)
ticker_val.dropna(axis=0, inplace=True)

idx_val, X_val, Y_val = fx.to_XY(ticker_val, elenco_features, elenco_targets, n_timesteps, giorni_previsione, addestramento=True)
X_val = X_val.reshape((-1, n_timesteps * n_features))
Y_val = Y_val.reshape((-1, giorni_previsione * n_targets))
X_scaler_val = clone(scaler)
Y_scaler_val = clone(scaler)
X_val = X_scaler_val.fit_transform(X_val)
Y_val = Y_scaler_val.fit_transform(Y_val)
X_val = X_val.reshape((-1, n_timesteps, n_features))
Y_val = Y_val.reshape((-1, giorni_previsione, n_targets))

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

for i_ticker in range (n_simboli_addestramento + 1):
    nome_simbolo = lista_ticker["Ticker"].iloc[i_ticker]
    print(f"\033[48;5;42m{i_ticker+1} di {n_simboli_addestramento}: Ticker {nome_simbolo}\033[0m")
    print("Download dati ticker")
    try:
        ticker = yf.download(nome_simbolo, start='2010-01-01', end='2023-12-31', progress=False)
        if ticker["Close"].iloc[-1] >= 1:
            ticker.index = ticker.index.date
            print("Calcolo indicatori ticker")
            ticker = fx.crea_indicatori(ticker)
            ticker.dropna(axis=0, inplace=True)

            print("Definizione features e target")
            idx, X, Y = fx.to_XY(ticker, elenco_features, elenco_targets, n_timesteps, giorni_previsione, addestramento=True)

            print("Scaler")
            X = X.reshape((-1, n_timesteps * n_features))
            Y = Y.reshape((-1, giorni_previsione * n_targets))

            #X_train, X_test, Y_train, Y_test, idx_train, idx_test = train_test_split(X, Y, idx, test_size=0.2, random_state=seed)
            X_train = X
            Y_train = Y

            X_scaler = clone(scaler)
            Y_scaler = clone(scaler)
            #X_scaler.fit(X_train)
            #Y_scaler.fit(Y_train)
            X_train = X_scaler.fit_transform(X_train)
            #X_test = X_scaler.fit_transform(X_test)
            Y_train = Y_scaler.fit_transform(Y_train)
            #Y_test = Y_scaler.fit_transform(Y_test)

            X_train = X_train.reshape((-1, n_timesteps, n_features))
            #X_test = X_test.reshape((-1, n_timesteps, n_features))
            Y_train = Y_train.reshape((-1, giorni_previsione, n_targets))
            #Y_test = Y_test.reshape((-1, giorni_previsione, n_targets))

            print(f"Addestramento modello epochs={epochs}, batch_size={batch_size}")
            model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, Y_val), callbacks=[early_stopping, reduce_lr])
            model.save("LSTM.keras")
    except Exception as e:
        print(e)
        continue


Download dati ticker validazione
Calcolo indicatori ticker validazione
1 di 2000: Ticker FIZZ
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
Addestramento modello epochs=100, batch_size=32
Epoch 1/100
103/103 [==============================] - 16s 96ms/step - loss: 20.6618 - val_loss: 48.7829 - lr: 0.0010
Epoch 2/100
103/103 [==============================] - 8s 79ms/step - loss: 17.3139 - val_loss: 55.0380 - lr: 0.0010
Epoch 3/100
103/103 [==============================] - 7s 72ms/step - loss: 17.9276 - val_loss: 54.3302 - lr: 0.0010
Epoch 4/100
103/103 [==============================] - 8s 76ms/step - loss: 16.0671 - val_loss: 56.7114 - lr: 0.0010
Epoch 5/100
103/103 [==============================] - 8s 74ms/step - loss: 16.8859 - val_loss: 54.0607 - lr: 0.0010
Epoch 6/100
103/103 [==============================] - 8s 74ms/step - loss: 15.9163 - val_loss: 57.2029 - lr: 0.0010
Epoch 7/100
103/103 [==============================] - 8s 73ms/step - l


1 Failed download:
['ENI']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2010-01-01 -> 2023-12-31)')


single positional indexer is out-of-bounds
5 di 2000: Ticker RNST
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
Addestramento modello epochs=100, batch_size=32
Epoch 1/100
103/103 [==============================] - 8s 76ms/step - loss: 25.5677 - val_loss: 48.7841 - lr: 1.0000e-04
Epoch 2/100
103/103 [==============================] - 8s 76ms/step - loss: 23.2522 - val_loss: 50.0197 - lr: 1.0000e-04
Epoch 3/100
103/103 [==============================] - 7s 73ms/step - loss: 22.6132 - val_loss: 51.8131 - lr: 1.0000e-04
Epoch 4/100
103/103 [==============================] - 7s 71ms/step - loss: 21.9175 - val_loss: 52.0628 - lr: 1.0000e-04
Epoch 5/100
103/103 [==============================] - 7s 72ms/step - loss: 21.8619 - val_loss: 53.4944 - lr: 1.0000e-04
Epoch 6/100
103/103 [==============================] - 8s 75ms/step - loss: 21.5895 - val_loss: 53.8010 - lr: 1.0000e-04
Epoch 7/100
103/103 [==============================] - 7s 71ms/step - loss: 

103/103 [==============================] - 7s 68ms/step - loss: 27.2444 - val_loss: 43.8553 - lr: 1.0000e-04
Epoch 13/100
103/103 [==============================] - 8s 74ms/step - loss: 26.9444 - val_loss: 44.5940 - lr: 1.0000e-04
Epoch 14/100
103/103 [==============================] - 7s 73ms/step - loss: 26.6815 - val_loss: 44.1018 - lr: 1.0000e-04
Epoch 15/100
103/103 [==============================] - 8s 76ms/step - loss: 26.4685 - val_loss: 43.9414 - lr: 1.0000e-04
Epoch 16/100
103/103 [==============================] - 9s 88ms/step - loss: 26.2848 - val_loss: 44.0214 - lr: 1.0000e-04
Epoch 17/100
103/103 [==============================] - 9s 88ms/step - loss: 26.2353 - val_loss: 43.5399 - lr: 1.0000e-04
10 di 2000: Ticker ATLC
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
Addestramento modello epochs=100, batch_size=32
Epoch 1/100
103/103 [==============================] - 7s 70ms/step - loss: 17.5262 - val_loss: 44.9479 - lr: 1.0000e-04
Epoc

Epoch 10/100
103/103 [==============================] - 7s 69ms/step - loss: 14.7307 - val_loss: 52.5397 - lr: 1.0000e-04
Epoch 11/100
103/103 [==============================] - 7s 68ms/step - loss: 14.7074 - val_loss: 54.4437 - lr: 1.0000e-04
15 di 2000: Ticker WOOF
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
Addestramento modello epochs=100, batch_size=32
Epoch 1/100
16/16 [==============================] - 3s 174ms/step - loss: 16.7605 - val_loss: 45.7866 - lr: 1.0000e-04
Epoch 2/100
16/16 [==============================] - 3s 171ms/step - loss: 16.1043 - val_loss: 45.8656 - lr: 1.0000e-04
Epoch 3/100
16/16 [==============================] - 3s 172ms/step - loss: 15.5759 - val_loss: 45.9196 - lr: 1.0000e-04
Epoch 4/100
16/16 [==============================] - 3s 171ms/step - loss: 15.7624 - val_loss: 45.9433 - lr: 1.0000e-04
Epoch 5/100
16/16 [==============================] - 3s 174ms/step - loss: 15.6264 - val_loss: 45.9700 - lr: 1.0000e-04


...


1 Failed download:
['TPRO']: Exception('%ticker%: No timezone found, symbol may be delisted')


single positional indexer is out-of-bounds
69 di 2000: Ticker ORI
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
Addestramento modello epochs=100, batch_size=32
Epoch 1/100
103/103 [==============================] - 7s 73ms/step - loss: 19.3322 - val_loss: 43.6249 - lr: 1.0000e-04
Epoch 2/100
103/103 [==============================] - 7s 72ms/step - loss: 17.7719 - val_loss: 43.9726 - lr: 1.0000e-04
Epoch 3/100
103/103 [==============================] - 8s 73ms/step - loss: 17.2019 - val_loss: 44.8752 - lr: 1.0000e-04
Epoch 4/100
103/103 [==============================] - 8s 73ms/step - loss: 16.8302 - val_loss: 45.2118 - lr: 1.0000e-04
Epoch 5/100
103/103 [==============================] - 8s 73ms/step - loss: 17.0797 - val_loss: 45.4218 - lr: 1.0000e-04
Epoch 6/100
103/103 [==============================] - 8s 75ms/step - loss: 16.5486 - val_loss: 45.3070 - lr: 1.0000e-04
Epoch 7/100
103/103 [==============================] - 8s 74ms/step - loss: 

11/11 [==============================] - 2s 236ms/step - loss: 11.5937 - val_loss: 47.3160 - lr: 1.0000e-04
Epoch 3/100
11/11 [==============================] - 3s 244ms/step - loss: 11.4451 - val_loss: 48.4892 - lr: 1.0000e-04
Epoch 4/100
11/11 [==============================] - 3s 247ms/step - loss: 11.2159 - val_loss: 48.5309 - lr: 1.0000e-04
Epoch 5/100
11/11 [==============================] - 2s 231ms/step - loss: 11.0070 - val_loss: 49.2629 - lr: 1.0000e-04
Epoch 6/100
11/11 [==============================] - 2s 237ms/step - loss: 10.9893 - val_loss: 49.8355 - lr: 1.0000e-04
Epoch 7/100
11/11 [==============================] - 2s 236ms/step - loss: 10.8936 - val_loss: 50.2270 - lr: 1.0000e-04
Epoch 8/100
11/11 [==============================] - 2s 234ms/step - loss: 10.5122 - val_loss: 50.4030 - lr: 1.0000e-04
Epoch 9/100
11/11 [==============================] - 2s 232ms/step - loss: 10.6435 - val_loss: 50.7880 - lr: 1.0000e-04
Epoch 10/100
11/11 [==============================] 

103/103 [==============================] - 7s 72ms/step - loss: 24.4691 - val_loss: 46.6890 - lr: 1.0000e-04
Epoch 6/100
103/103 [==============================] - 7s 72ms/step - loss: 24.0175 - val_loss: 46.7373 - lr: 1.0000e-04
Epoch 7/100
103/103 [==============================] - 8s 74ms/step - loss: 24.8527 - val_loss: 45.9500 - lr: 1.0000e-04
Epoch 8/100
103/103 [==============================] - 8s 75ms/step - loss: 24.1023 - val_loss: 46.6495 - lr: 1.0000e-04
Epoch 9/100
103/103 [==============================] - 8s 74ms/step - loss: 23.6588 - val_loss: 47.2191 - lr: 1.0000e-04
Epoch 10/100
103/103 [==============================] - 7s 72ms/step - loss: 23.4051 - val_loss: 46.4873 - lr: 1.0000e-04
Epoch 11/100
103/103 [==============================] - 7s 73ms/step - loss: 23.2481 - val_loss: 47.0012 - lr: 1.0000e-04
Epoch 12/100
103/103 [==============================] - 7s 72ms/step - loss: 23.2176 - val_loss: 46.9395 - lr: 1.0000e-04
80 di 2000: Ticker PLTK
Download dati tic

Epoch 8/100
103/103 [==============================] - 8s 73ms/step - loss: 11.0874 - val_loss: 57.6370 - lr: 1.0000e-04
Epoch 9/100
103/103 [==============================] - 8s 74ms/step - loss: 10.9143 - val_loss: 59.3010 - lr: 1.0000e-04
Epoch 10/100
103/103 [==============================] - 7s 73ms/step - loss: 10.9046 - val_loss: 59.6491 - lr: 1.0000e-04
Epoch 11/100
103/103 [==============================] - 8s 76ms/step - loss: 10.7528 - val_loss: 59.6766 - lr: 1.0000e-04
85 di 2000: Ticker SSRM
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
Addestramento modello epochs=100, batch_size=32
Epoch 1/100
103/103 [==============================] - 7s 72ms/step - loss: 20.9364 - val_loss: 48.0412 - lr: 1.0000e-04
Epoch 2/100
103/103 [==============================] - 7s 72ms/step - loss: 19.6407 - val_loss: 47.2527 - lr: 1.0000e-04
Epoch 3/100
103/103 [==============================] - 7s 72ms/step - loss: 19.3977 - val_loss: 46.5942 - lr: 1.0000

In [ ]:

print("Download dati ticker per previsione")
dati_previsione = yf.download(simbolo_test, start='2010-01-01', end='2023-12-31', progress=False)
dati_previsione.index = dati_previsione.index.date
dati_previsione = fx.crea_indicatori(dati_previsione)
dati_previsione.iloc[:100] = dati_previsione.iloc[:100].dropna(axis=0)

indice_prev, X_prev, Y_prev = fx.to_XY(dati_previsione, elenco_features, elenco_targets, n_timesteps, giorni_previsione, addestramento=False)

X_prev = X_prev.reshape(-1, n_timesteps * n_features)
Y_prev = Y_prev.reshape(-1, giorni_previsione * n_targets)
X_scaler_prev = clone(scaler)
Y_scaler_prev = clone(scaler)
X_prev = X_scaler_prev.fit_transform(X_prev)
Y_prev = Y_scaler_prev.fit_transform(Y_prev)
X_prev = X_prev.reshape(-1, n_timesteps, n_features)
Y_prev = Y_prev.reshape(-1, giorni_previsione, n_targets)

pred = model.predict(X_prev)
#pred_loss, pred_mae = model.evaluate(X_prev, Y_prev)

X_prev = X_prev.reshape(-1, n_timesteps * n_features)
X_prev = X_scaler_prev.inverse_transform(X_prev)
X_prev = X_prev.reshape(-1, n_timesteps, n_features)

pred = pred.reshape(-1, giorni_previsione * n_targets)
pred = Y_scaler_prev.inverse_transform(pred)
pred = pred.reshape(-1, giorni_previsione, n_targets)

col_analisi = "EMA_5"

import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.subplots as sp
import yfinance as yf

def grafico1(dati_previsione, pred):
    dati_previsione = dati_previsione.loc[indice_prev]
    new_dates = pd.bdate_range(start=dati_previsione.index[-1] + pd.Timedelta(days=1), periods=giorni_previsione)
    df_new = pd.DataFrame(index=new_dates.date)
    dati_previsione = pd.concat([dati_previsione, df_new])
    risultato = pd.DataFrame({"Previsione": pred[:, giorni_previsione-1, col_targets[col_analisi]].round(2)}, index=dati_previsione.index[giorni_previsione:])
    risultato = pd.concat([dati_previsione[col_analisi], risultato], axis=1)
    
    target = go.Scatter(
        x = risultato.index,
        y = risultato[col_analisi],
        mode = 'lines',
        line = dict(color='rgba(0, 0, 0, .9)'),
        name = col_analisi
    )

    previsione = go.Scatter(
        x = risultato.index,
        y = risultato['Previsione'],
        mode = 'lines',
        line = dict(color='rgba(0, 0, 250, .9)'),
        name = 'Previsione'
    )
    '''
    err_meno = go.Scatter(
        x = risultato.index,
        y = risultato['Previsione'] - err, 
        mode = 'lines',
        line = dict(
            color='rgba(0, 0, 250, .9)',
            width=1
        ),
        connectgaps = False,
        name = 'err-'
    )                
    err_piu = go.Scatter(
        x = risultato.index,
        y = risultato['Previsione'] + err, 
        mode = 'lines',
        line = dict(
            color='rgba(0, 0, 250, .9)',
            width=1
        ),
        fill='tonexty',
        fillcolor='rgba(0, 0, 250, .2)', # Puoi cambiare il valore di alpha per regolare la trasparenza
        connectgaps = False,
        name = 'err+'
    )
    '''
    layout = dict(xaxis = dict(autorange=True),
                  yaxis = dict(title = 'Previsioni', autorange=True),
                  autosize = True,
                  margin = go.layout.Margin(
                      l=0,  # Sinistra
                      r=0,  # Destra
                      b=0,  # Basso
                      t=50,  # Alto
                      pad=0  # Padding
                  ),
                  legend = dict(traceorder = 'normal', bordercolor = 'black')
    )
    fig = sp.make_subplots(rows=1, cols=1, shared_xaxes=True)
    fig.update_layout(layout)
    fig.add_trace(target, row=1, col=1)
    fig.add_trace(previsione, row=1, col=1)
    #fig.add_trace(err_meno, row=1, col=1)
    #fig.add_trace(err_piu, row=1, col=1)
    pyo.plot(fig, filename="grafico1.html", auto_open=True)




In [ ]:
def grafico2(dati_previsione, pred):
    dati2 = dati_previsione.dropna(axis=0).iloc[-60:]
    new_dates = pd.bdate_range(start=dati2.index[-1] + pd.Timedelta(days=1), periods=giorni_previsione)
    df_new = pd.DataFrame(index=new_dates.date)

    val_prev2 = (pred[-1, i, col_targets[col_analisi]] for i in range(giorni_previsione))
    prev2 = pd.DataFrame(val_prev2, columns=["Previsione"], index=df_new.index)
    riga_iniziale = pd.DataFrame({"Previsione": [dati2[col_analisi].iloc[-1]]}, index=[dati2.index[-1]])
    prev2 = pd.concat([riga_iniziale, prev2], axis=0)
    
    target = go.Scatter(
        x = dati2.index,
        y = dati2[col_analisi].round(2),
        mode = 'lines',
        line = dict(color='rgba(0, 0, 0, .9)'),
        name = col_analisi
    )

    previsione = go.Scatter(
        x = prev2.index,
        y = prev2['Previsione'].round(2),
        mode = 'lines',
        line = dict(color='rgba(0, 0, 250, .9)'),
        name = 'Previsione'
    )
    layout = dict(xaxis = dict(autorange=True),
                  yaxis = dict(title = 'Previsioni', autorange=True),
                  autosize = True,
                  margin = go.layout.Margin(
                      l=0,  # Sinistra
                      r=0,  # Destra
                      b=0,  # Basso
                      t=50,  # Alto
                      pad=0  # Padding
                  ),
                  legend = dict(traceorder = 'normal', bordercolor = 'black')
    )
    fig = sp.make_subplots(rows=1, cols=1, shared_xaxes=True)
    fig.update_layout(layout)
    fig.add_trace(target, row=1, col=1)
    fig.add_trace(previsione, row=1, col=1)
    pyo.plot(fig, filename="grafico2.html", auto_open=True)



In [ ]:
grafico1(dati_previsione, pred)

In [ ]:
grafico2(dati_previsione, pred)